REFERENCIA
https://medium.com/mlearning-ai/neat-data-preprocessing-with-pipeline-and-columntransformer-2a0468865b6b

In [56]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, MaxAbsScaler, LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.tree import plot_tree
from sklearn.compose import ColumnTransformer

https://data.world/data-society/bank-marketing-data

In [14]:
source = 'https://raw.githubusercontent.com/nparis87/MachineLearning/main/bank-additional-full.csv'

data = pd.read_csv(source, sep = ';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Otra forma de entender los datos

In [20]:
for col in data.columns:
    print(col, '\n', data[col].unique(), '\n\n')

age 
 [56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89] 


job 
 ['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student'] 


marital 
 ['married' 'single' 'divorced' 'unknown'] 


education 
 ['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate'] 


default 
 ['no' 'unknown' 'yes'] 


housing 
 ['no' 'yes' 'unknown'] 


loan 
 ['no' 'yes' 'unknown'] 


contact 
 ['telephone' 'cellular'] 


month 
 ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep'] 


day_of_week 
 ['mon' 'tue' 'wed' 'thu' 'fri'] 


duration 
 [ 261  149  226 ... 1246 1556 1868] 


campaign 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 14 22 25 16 17 15 20 56
 39 35 42

## TRANSFORMACIÓN DE DATOS 2.0

In [75]:
num_cols = ['duration'] 
cat_cols = ['contact'] #'codificado simple'
cat_cols_dummy = ['day_of_week'] # One hot encoder

In [76]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale',MinMaxScaler())])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot',OrdinalEncoder())])

cat_dummy_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('categorical', OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [77]:
# TRANSFORMACIÓN POR COLUMNAS

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols),
    ('cat_dummy', cat_dummy_pipeline, cat_cols_dummy)
    ],
    remainder='drop',
    n_jobs=-1)

In [78]:
X = data.drop('y', axis = 1)
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [79]:
model = DecisionTreeClassifier()
#vc_tree = cross_validate(model, X, y, cv=5, scoring=('accuracy'), return_train_score=True)

model_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', model)
])

In [80]:
from sklearn import set_config
set_config(display='diagram')
display(model_pipeline)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['duration']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OrdinalEncoder())]),
                                                  ['contact']),
                                                 ('cat_dummy',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('categorical',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['day_of_week'])])),
                ('model', DecisionTreeClassifier())])

In [81]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['duration']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OrdinalEncoder())]),
                                                  ['contact']),
                                                 ('cat_dummy',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('categorical',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['day_of_week'])])),
                ('model', DecisionTreeClassifier())])

In [84]:
model_pipeline.predict(X_test)

array(['no', 'no', 'yes', ..., 'no', 'no', 'no'], dtype=object)

## Trabajo en clase, ver reporte de puntajes y matriz de confusión